In [1]:
import tensorflow as tf
import sionna as sn
import numpy as np
import cv2, os, math, time
from PIL import Image

In [2]:
# 이미지 폴더 경로
image_folder = "images"
output_folder = "resized_images"

# 출력 폴더가 없으면 생성
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 폴더 내의 파일을 순회하면서 처리
for filename in os.listdir(image_folder):
    if filename.endswith(".jpg"):  # JPG 파일만 처리
        img_path = os.path.join(image_folder, filename)
        img = Image.open(img_path)
        
        # 이미지 리사이즈 (256x256)
        img_resized = img.resize((256, 256))
        
        # 파일명과 확장자 분리
        file_base = os.path.splitext(filename)[0]
        
        # PNG로 저장
        output_path = os.path.join(output_folder, f"{file_base}.png")
        img_resized.save(output_path, "PNG")

print(f"모든 이미지가 256x256으로 리사이즈되고 PNG로 변환되었습니다.")

모든 이미지가 256x256으로 리사이즈되고 PNG로 변환되었습니다.


In [3]:
resized_folder = "resized_images"
output_folder = "output_j2k"

# 출력 폴더가 없으면 생성
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# opj_compress.exe 경로
compress_exe = ".\\opj_compress.exe"

# 모든 PNG 파일을 순회하면서 J2K로 변환
for filename in os.listdir(resized_folder):
    if filename.endswith(".png"):  # PNG 파일만 처리
        input_path = os.path.join(resized_folder, filename)
        output_filename = filename.replace(".png", ".j2k")
        output_path = os.path.join(output_folder, output_filename)
        
        # opj_compress 명령어 실행
        command = f'{compress_exe} -i {input_path} -o {output_path}'
        os.system(command)

print(f"모든 이미지가 J2K 형식으로 변환되었습니다.")

모든 이미지가 J2K 형식으로 변환되었습니다.


In [9]:
class RealAWGNSignalTransmission(tf.keras.Model):
    def __init__(self, SNR_dB, shape, bw):
        super(RealAWGNSignalTransmission, self).__init__()
        
        # SNR을 Eb/N0에서 노이즈 전력으로 변환
        self.noise = sn.utils.ebnodb2no(SNR_dB, num_bits_per_symbol=4, coderate=3/4)
        self.channel = sn.channel.AWGN()
        self.k = np.prod(shape) * bw  # 전체 데이터 비트 수 계산

    def encoderNormalizer(self, z):
        norm = tf.math.sqrt(tf.reduce_sum(tf.math.conj(z) * z))
        return z * math.sqrt(self.k) / norm  # Assume power constraint is 1

    def call(self, inputs):
        """
        입력 데이터에 AWGN 채널을 적용한 후 전송합니다.
        :param inputs: 전송할 이미지 데이터
        :return: 노이즈가 추가된 신호
        """
        # 정규화된 신호 적용
        normalized_signal = self.encoderNormalizer(inputs)
        transmitted_signal = self.channel([normalized_signal, self.noise])  # AWGN 채널 적용
        
        # 데이터 값 클리핑 (0-255 사이)
#         transmitted_signal = tf.clip_by_value(transmitted_signal, 0, 255)
        return transmitted_signal

In [10]:
SNR_values = [1, 4, 7, 10, 13, 16, 19, 22, 25]
bw = 1/6  # 대역폭 압축 비율 예시

In [11]:
# 경로와 파라미터 설정
input_folder = "output_j2k"  # 원본 j2k 파일 폴더 경로
output_base_folder = "noisy_output_j2k"  # 노이즈 추가 후 저장할 폴더 경로

# SNR별 출력 폴더 생성
for snr in SNR_values:
    os.makedirs(f"{output_base_folder}/SNR_{snr}", exist_ok=True)

# 각 이미지 파일 처리
for filename in os.listdir(input_folder):
    if filename.endswith(".j2k"):
        # 이미지 읽기
        image_path = os.path.join(input_folder, filename)
        image_data = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        image_data = cv2.cvtColor(image_data, cv2.COLOR_BGR2RGB)
        image_tensor = tf.convert_to_tensor(image_data, dtype=tf.complex64)
        shape = image_tensor.shape

        # 각 SNR에 따른 노이즈 추가 및 저장
        for snr in SNR_values:
            # 모델 실행 및 시간 측정 시작 (데이터양 및 전송시간 측정 시 사용)
#             start_time = time.time()
            
            model = RealAWGNSignalTransmission(SNR_dB=snr, shape=shape, bw=bw)
            noisy_image = model(image_tensor)
            noisy_image_real = tf.math.real(noisy_image)
            noisy_image_abs = tf.abs(noisy_image_real)
            noisy_image_scaled = noisy_image_abs * 255
            noisy_image_clipped = tf.clip_by_value(noisy_image_scaled, 0, 255)
            noisy_image_np = noisy_image_clipped.numpy().astype(np.uint8)
            
            end_time = time.time()    # 모델 실행 및 시간 측정 끝

#             # 전송 시간 계산
#             transmission_time = end_time - start_time

#             # 데이터 패킷 크기 측정
#             shape_ = noisy_image.shape
#             original_data_size = shape_[0] * shape_[1] * shape_[2] * 32  # 비트 단위
#             compressed_data_size = tf.size(noisy_image).numpy()  # 압축된 데이터 크기 complex64이기 때문에

#             # 결과 출력
#             print(f"Original Data Size: {original_data_size / 8 / 1024:.2f} KB")  # KB 단위로 변환
#             print(f"Compressed Data Size: {compressed_data_size / 8 / 1024:.2f} KB")
#             print(f"Transmission Time: {transmission_time:.2f} seconds")

#             # 데이터 전송률 계산 (Throughput)
#             throughput = compressed_data_size / transmission_time  # 데이터 전송률 (bps)
#             print(f"Data Throughput: {throughput / 10**6:.2f} Mbps")  # Mbps로 변환
            
            # 각 SNR 폴더에 이미지 저장
            output_path = f"{output_base_folder}/SNR_{snr}/{filename}"
            image_pil = Image.fromarray(noisy_image_np)
            image_pil.save(output_path, format="JPEG2000")
            
            print(f"SNR {snr}로 노이즈가 추가된 이미지를 {output_path}에 저장했습니다.")

SNR 1로 노이즈가 추가된 이미지를 noisy_output_j2k/SNR_1/0000011_05068_d_0000008.j2k에 저장했습니다.
SNR 4로 노이즈가 추가된 이미지를 noisy_output_j2k/SNR_4/0000011_05068_d_0000008.j2k에 저장했습니다.
SNR 7로 노이즈가 추가된 이미지를 noisy_output_j2k/SNR_7/0000011_05068_d_0000008.j2k에 저장했습니다.
SNR 10로 노이즈가 추가된 이미지를 noisy_output_j2k/SNR_10/0000011_05068_d_0000008.j2k에 저장했습니다.
SNR 13로 노이즈가 추가된 이미지를 noisy_output_j2k/SNR_13/0000011_05068_d_0000008.j2k에 저장했습니다.
SNR 16로 노이즈가 추가된 이미지를 noisy_output_j2k/SNR_16/0000011_05068_d_0000008.j2k에 저장했습니다.
SNR 19로 노이즈가 추가된 이미지를 noisy_output_j2k/SNR_19/0000011_05068_d_0000008.j2k에 저장했습니다.
SNR 22로 노이즈가 추가된 이미지를 noisy_output_j2k/SNR_22/0000011_05068_d_0000008.j2k에 저장했습니다.
SNR 25로 노이즈가 추가된 이미지를 noisy_output_j2k/SNR_25/0000011_05068_d_0000008.j2k에 저장했습니다.
SNR 1로 노이즈가 추가된 이미지를 noisy_output_j2k/SNR_1/0000011_06000_d_0000009.j2k에 저장했습니다.
SNR 4로 노이즈가 추가된 이미지를 noisy_output_j2k/SNR_4/0000011_06000_d_0000009.j2k에 저장했습니다.
SNR 7로 노이즈가 추가된 이미지를 noisy_output_j2k/SNR_7/0000011_06000_d_0000009.j2k에 저장했습니다.
SNR 10로 노이즈가 추가된

SNR 7로 노이즈가 추가된 이미지를 noisy_output_j2k/SNR_7/0000103_02544_d_0000029.j2k에 저장했습니다.
SNR 10로 노이즈가 추가된 이미지를 noisy_output_j2k/SNR_10/0000103_02544_d_0000029.j2k에 저장했습니다.
SNR 13로 노이즈가 추가된 이미지를 noisy_output_j2k/SNR_13/0000103_02544_d_0000029.j2k에 저장했습니다.
SNR 16로 노이즈가 추가된 이미지를 noisy_output_j2k/SNR_16/0000103_02544_d_0000029.j2k에 저장했습니다.
SNR 19로 노이즈가 추가된 이미지를 noisy_output_j2k/SNR_19/0000103_02544_d_0000029.j2k에 저장했습니다.
SNR 22로 노이즈가 추가된 이미지를 noisy_output_j2k/SNR_22/0000103_02544_d_0000029.j2k에 저장했습니다.
SNR 25로 노이즈가 추가된 이미지를 noisy_output_j2k/SNR_25/0000103_02544_d_0000029.j2k에 저장했습니다.
SNR 1로 노이즈가 추가된 이미지를 noisy_output_j2k/SNR_1/0000115_00315_d_0000080.j2k에 저장했습니다.
SNR 4로 노이즈가 추가된 이미지를 noisy_output_j2k/SNR_4/0000115_00315_d_0000080.j2k에 저장했습니다.
SNR 7로 노이즈가 추가된 이미지를 noisy_output_j2k/SNR_7/0000115_00315_d_0000080.j2k에 저장했습니다.
SNR 10로 노이즈가 추가된 이미지를 noisy_output_j2k/SNR_10/0000115_00315_d_0000080.j2k에 저장했습니다.
SNR 13로 노이즈가 추가된 이미지를 noisy_output_j2k/SNR_13/0000115_00315_d_0000080.j2k에 저장했습니다.
SNR 16로 노이즈가

In [12]:
# SNR별 출력 폴더 생성
for snr in SNR_values:
    os.makedirs(f"result/16-QAM/SNR_{snr}", exist_ok=True)

# 각 SNR 폴더의 j2k 파일을 PNG로 변환
for snr in SNR_values:
    input_folder = f"noisy_output_j2k/SNR_{snr}"  # 각 SNR 폴더 경로
    output_folder = f"result/16-QAM/SNR_{snr}"  # 변환된 PNG 저장 경로
    
    # opj_compress.exe 경로
    decompress_exe = ".\\opj_decompress.exe"
    
    for filename in os.listdir(input_folder):
        if filename.endswith(".j2k"):  # j2k 파일만 처리
            input_path = os.path.join(input_folder, filename)
            output_filename = filename.replace(".j2k", ".png")
            output_path = os.path.join(output_folder, output_filename)

            # opj_compress 명령어 실행
            command = f'{decompress_exe} -i {input_path} -o {output_path}'
            os.system(command)
            
    print(f"{input_folder}를 {output_folder}로 이미지 변환하여 저장했습니다.")

noisy_output_j2k/SNR_1를 result/16-QAM/SNR_1로 이미지 변환하여 저장했습니다.
noisy_output_j2k/SNR_4를 result/16-QAM/SNR_4로 이미지 변환하여 저장했습니다.
noisy_output_j2k/SNR_7를 result/16-QAM/SNR_7로 이미지 변환하여 저장했습니다.
noisy_output_j2k/SNR_10를 result/16-QAM/SNR_10로 이미지 변환하여 저장했습니다.
noisy_output_j2k/SNR_13를 result/16-QAM/SNR_13로 이미지 변환하여 저장했습니다.
noisy_output_j2k/SNR_16를 result/16-QAM/SNR_16로 이미지 변환하여 저장했습니다.
noisy_output_j2k/SNR_19를 result/16-QAM/SNR_19로 이미지 변환하여 저장했습니다.
noisy_output_j2k/SNR_22를 result/16-QAM/SNR_22로 이미지 변환하여 저장했습니다.
noisy_output_j2k/SNR_25를 result/16-QAM/SNR_25로 이미지 변환하여 저장했습니다.


In [27]:
# 원본 및 통신 완료된 이미지의 PSNR 구하기

def calculate_psnr(original, restored):
    mse = np.mean((original - restored) ** 2)
    if mse == 0:  # MSE가 0이면 완벽히 동일한 이미지이므로 PSNR은 무한대
        return float('inf')
    max_pixel_value = 255.0
    psnr = 20 * np.log10(max_pixel_value / np.sqrt(mse))
    return psnr

# 원본 이미지와 복원 이미지가 저장된 폴더 경로
original_folder = "resized_images"
restored_folder = "result_U-Net/SNR_19"

# 두 폴더에 있는 파일들을 차례대로 비교
for filename in os.listdir(original_folder):
    if filename.endswith(".png"):  # PNG 파일만 처리
        original_image_path = os.path.join(original_folder, filename)
        restored_image_path = os.path.join(restored_folder, filename)
        
        # 이미지 읽기
        original_image = cv2.imread(original_image_path, cv2.IMREAD_UNCHANGED)
        restored_image = cv2.imread(restored_image_path, cv2.IMREAD_UNCHANGED)
        
        # PSNR 계산
        psnr_value = calculate_psnr(original_image, restored_image)
        print(f"{psnr_value}")

29.456804231478102
29.244816686869367
29.796913000197204
29.80402367540778
29.70188900803649
33.049418510790005
29.582955683278126
29.41054983593017
30.05768127753701
29.743634742594214
29.71491342698013
29.707841285362004
31.728687101645647
31.979867762261183
29.612776795872563
29.138231461502222
31.272176258675426
29.01096934967411
29.71238509083433
29.779249071477693
29.477495368308304
